# Interactive Germany Bridge Map

- Sidebar filters: layers, material class, year range, road type, choropleth metric
- Choropleth: mean condition / load index / age / substance per county
- Points: color-coded by condition (green = good → red = bad)

Open: http://127.0.0.1:8071


In [2]:
import re
import json
import unicodedata
from pathlib import Path

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import plotly.graph_objects as go

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from shapely.geometry import mapping
from tueplots import bundles

geo_path = Path("../../data/districts.geojson")
states_path = Path("../../data/states.geojson")
csv_path = Path("../../data/original_bridge_statistic_germany.csv")

AGE_REF_YEAR = 2024


## Helper functions

In [3]:
def pick_first(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None


def norm_name(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("ä", "ae").replace("ö", "oe").replace("ü", "ue").replace("ß", "ss")
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"\b(landkreis|kreisfreie stadt|kreis|stadt|region|lkr\.?)\b", "", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()


def make_points_from_lonlat(dfin, lon_col="x2", lat_col="y2"):
    lon = pd.to_numeric(dfin[lon_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    lat = pd.to_numeric(dfin[lat_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    ok = lon.notna() & lat.notna()
    g = gpd.GeoDataFrame(
        dfin[ok].copy(),
        geometry=[Point(xy) for xy in zip(lon[ok], lat[ok])],
        crs=4326,
    )
    return g


def make_points_from_utm(dfin, x_col="X", y_col="Y", epsg=25832):
    xx = pd.to_numeric(dfin[x_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    yy = pd.to_numeric(dfin[y_col].astype(str).str.replace(",", ".", regex=False), errors="coerce")
    ok = xx.notna() & yy.notna()
    g = gpd.GeoDataFrame(
        dfin[ok].copy(),
        geometry=[Point(xy) for xy in zip(xx[ok], yy[ok])],
        crs=epsg,
    ).to_crs(4326)
    return g


def safe_minmax(series: pd.Series):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if len(s) == 0:
        return None
    vmin = float(s.min())
    vmax = float(s.max())
    if vmin == vmax:
        eps = 1e-6 if vmin == 0 else abs(vmin) * 1e-6
        vmin -= eps
        vmax += eps
    return vmin, vmax


def safe_minmax_with_fallback(series: pd.Series, fallback=(1.0, 4.0)):
    rng = safe_minmax(series)
    if rng is None:
        return fallback
    return rng

## Load GeoJSON (counties + states of germany)

In [4]:
gdf_kreise = gpd.read_file(geo_path)

colsK = gdf_kreise.columns
col_name = pick_first(colsK, ["GEN", "NAME_3", "NAME", "GEN_NAME", "KREIS"])
col_ags = pick_first(colsK, ["AGS", "AGS_0", "RS", "RS_0", "ID_3"])

if col_name is None:
    raise ValueError(f"Could not find a county name column. Columns: {list(colsK)}")

if gdf_kreise.crs is None:
    gdf_kreise = gdf_kreise.set_crs(4326)
else:
    gdf_kreise = gdf_kreise.to_crs(4326)

gdf_states = gpd.read_file(states_path)
if gdf_states.crs is None:
    gdf_states = gdf_states.set_crs(4326)
else:
    gdf_states = gdf_states.to_crs(4326)

colsS = gdf_states.columns
state_name_col = pick_first(colsS, ["GEN", "NAME", "STATE_NAME"])
state_id_col = pick_first(colsS, ["RS", "RS_0", "AGS", "AGS_0", "ID"])
if state_id_col is None:
    state_id_col = state_name_col
gdf_states["state_id"] = gdf_states[state_id_col].astype(str)

## Load bridge CSV & clean columns

In [5]:
df = pd.read_csv(csv_path, sep=";", decimal=",", dtype=str, keep_default_na=False)

for sp in [
    "Zustandsnote", "Substanzkennzahl",
    "x2", "y2", "X", "Y",
    "Kreis", "Baujahr Überbau", "Traglastindex",
    "Zugeordneter Sachverhalt vereinfacht", "Baustoffklasse", "Bauwerksname"
]:
    if sp not in df.columns:
        df[sp] = ""

df["Zustandsnote"] = pd.to_numeric(
    df["Zustandsnote"].astype(str).str.replace(",", ".", regex=False),
    errors="coerce"
)

df["Substanzkennzahl"] = pd.to_numeric(
    df["Substanzkennzahl"].astype(str).str.replace(",", ".", regex=False),
    errors="coerce"
)

df["Baujahr Überbau"] = pd.to_numeric(
    df["Baujahr Überbau"].astype(str).str.replace(",", ".", regex=False),
    errors="coerce"
)


## Create bridge points

In [6]:
gdf_pts = make_points_from_lonlat(df, "x2", "y2")
used_coords = "WGS84 (x2/y2)"
if len(gdf_pts) == 0:
    gdf_pts = make_points_from_utm(df, "X", "Y", epsg=25832)
    used_coords = "UTM32 (EPSG:25832)"

gdf_pts["lon"] = gdf_pts.geometry.x
gdf_pts["lat"] = gdf_pts.geometry.y
gdf_pts["Baujahr"] = pd.to_numeric(gdf_pts["Baujahr Überbau"], errors="coerce")

spatial_join_ok = len(gdf_pts) > 0
if spatial_join_ok:
    joined = gpd.sjoin(
        gdf_pts,
        gdf_kreise[[col_name, col_ags, "geometry"]] if col_ags is not None else gdf_kreise[[col_name, "geometry"]],
        how="left",
        predicate="within",
    )
else:
    joined = None


## County name mapping 

In [7]:
kreise_names = gdf_kreise[[col_name]].copy()
kreise_names["__kreis_norm__"] = kreise_names[col_name].map(norm_name)

base = joined if spatial_join_ok else df.copy()

base["__br_kreis_norm__"] = base["Kreis"].map(norm_name)
name_map = dict(zip(kreise_names["__kreis_norm__"], gdf_kreise[col_name]))
map_series = base["__br_kreis_norm__"].map(name_map)

if spatial_join_ok:
    if col_name in base.columns:
        base[col_name] = base[col_name].fillna(map_series)
    else:
        base[col_name] = map_series
else:
    base[col_name] = map_series

## Feature engineering and aggregation per county

In [8]:
roman_map = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5}
base["Traglastindex_norm"] = (
    base["Traglastindex"]
    .astype(str)
    .str.upper()
    .str.strip()
    .map(roman_map)
)

base["Alter_Überbau"] = AGE_REF_YEAR - base["Baujahr Überbau"]

agg = (
    base
    .dropna(subset=[col_name])
    .groupby(col_name, as_index=False)
    .agg(
        n_bridges=("Bauwerksname", "size"),
        mean_zust=("Zustandsnote", "mean"),
        mean_trag=("Traglastindex_norm", "mean"),
        mean_age=("Alter_Überbau", "mean"),
        mean_substanz=("Substanzkennzahl", "mean"),
        median_zust=("Zustandsnote", "median"),
    )
)

if col_ags is not None:
    kreise_keys = gdf_kreise[[col_name, col_ags]].drop_duplicates()
    agg = agg.merge(kreise_keys, on=col_name, how="left")
    agg = agg[[col_ags, col_name, "n_bridges", "mean_zust", "mean_trag", "mean_age", "mean_substanz", "median_zust"]].rename(
        columns={col_name: "NAME"}
    )
else:
    agg = agg[[col_name, "n_bridges", "mean_zust", "mean_trag", "mean_age", "mean_substanz", "median_zust"]].rename(
        columns={col_name: "NAME"}
    )

agg = agg.sort_values("NAME").reset_index(drop=True)


## Build GeoJSON objects for Plotly

In [9]:
gplot = gdf_kreise.merge(agg, left_on=col_name, right_on="NAME", how="left")

id_col = None
if col_ags is not None:
    for c in [col_ags, f"{col_ags}_x", f"{col_ags}_y"]:
        if c in gplot.columns:
            id_col = c
            break

if id_col is not None:
    gplot["id"] = gplot[id_col].astype(str)
else:
    gplot["id"] = gplot["NAME"].astype(str)

geojson_kreise = json.loads(gplot[["id", "geometry"]].to_json())
geojson_states = json.loads(gdf_states[["state_id", "geometry"]].to_json())

germany_outline = gdf_states.dissolve().reset_index(drop=True)
germany_outline["id"] = "germany_outline"
outline_json = json.loads(germany_outline[["id", "geometry"]].to_json())


## Hover text + filter options

In [10]:
zust = pd.to_numeric(gdf_pts["Zustandsnote"], errors="coerce")
subs = pd.to_numeric(gdf_pts["Substanzkennzahl"], errors="coerce")

mapping_strasse = {"B": "Federal road", "O": "Federal road", "A": "Motorway"}
strassen_roh = gdf_pts["Zugeordneter Sachverhalt vereinfacht"].astype(str)
gdf_pts["RoadTypeLabel"] = strassen_roh.map(lambda x: mapping_strasse.get(x, x))

gdf_pts["hover"] = (
    "<b>" + gdf_pts["Bauwerksname"].astype(str) + "</b><br>"
    "Condition: " + zust.round(2).astype(str) + "<br>"
    "Substance score: " + subs.round(2).astype(str) + "<br>"
    "Material class: " + gdf_pts["Baustoffklasse"].astype(str) + "<br>"
    "Year (superstructure): " + gdf_pts["Baujahr Überbau"].astype(str) + "<br>"
    "Road type: " + gdf_pts["RoadTypeLabel"].astype(str)
)

layer_options = [
    {"label": "Outline", "value": "outline"},
    {"label": "States", "value": "states"},
    {"label": "Condition / Load / Age / Substance", "value": "choropleth"},
    {"label": "County borders", "value": "kreise"},
]

baustoff_vals = sorted(gdf_pts["Baustoffklasse"].dropna().unique())
baustoff_options = [{"label": b, "value": b} for b in baustoff_vals]

road_vals = sorted(gdf_pts["RoadTypeLabel"].dropna().unique())
road_options = [{"label": r, "value": r} for r in road_vals]

## Slider ranges & choropleth mode

In [11]:
BAUJAHR_MIN = 1900
if gdf_pts["Baujahr"].notna().any():
    BAUJAHR_MAX_DATA = int(gdf_pts["Baujahr"].max())
else:
    BAUJAHR_MAX_DATA = AGE_REF_YEAR

rest = (BAUJAHR_MAX_DATA - BAUJAHR_MIN) % 25
BAUJAHR_MAX = BAUJAHR_MAX_DATA + (25 - rest) if rest != 0 else BAUJAHR_MAX_DATA
baujahr_marks = {year: str(year) for year in range(BAUJAHR_MIN, BAUJAHR_MAX + 1, 25)}

choropleth_mode_options = [
    {"label": "Condition", "value": "zustand"},
    {"label": "Load index", "value": "trag"},
    {"label": "Mean age", "value": "alter"},
    {"label": "Substance score", "value": "substanz"},
]

## Figure

In [15]:
def build_figure(selected_layers, selected_baustoff, selected_year_range, selected_road, choropleth_mode):
    fig = go.Figure()

    if "choropleth" in selected_layers:
        if choropleth_mode == "trag":
            metric_col = "mean_trag"
            metric_label = "Mean load index"
            fmt = ".2f"
        elif choropleth_mode == "alter":
            metric_col = "mean_age"
            metric_label = "Mean age (years)"
            fmt = ".1f"
        elif choropleth_mode == "substanz":
            metric_col = "mean_substanz"
            metric_label = "Mean substance score"
            fmt = ".2f"
        else:
            metric_col = "mean_zust"
            metric_label = "Mean condition"
            fmt = ".2f"

        pts_scale = gdf_pts.copy()

        if selected_baustoff:
            pts_scale = pts_scale[pts_scale["Baustoffklasse"].isin(selected_baustoff)]

        if selected_year_range is not None and isinstance(selected_year_range, (list, tuple)) and len(selected_year_range) == 2:
            year_min, year_max = selected_year_range
            pts_scale = pts_scale[(pts_scale["Baujahr"].notna()) & (pts_scale["Baujahr"] >= year_min) & (pts_scale["Baujahr"] <= year_max)]

        if selected_road:
            pts_scale = pts_scale[pts_scale["RoadTypeLabel"].isin(selected_road)]

        if choropleth_mode == "trag":
            roman_map_local = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5}
            pts_metric = pts_scale["Traglastindex"].astype(str).str.upper().str.strip().map(roman_map_local)
        elif choropleth_mode == "alter":
            bauj = pd.to_numeric(pts_scale["Baujahr Überbau"], errors="coerce")
            pts_metric = AGE_REF_YEAR - bauj
        elif choropleth_mode == "substanz":
            pts_metric = pd.to_numeric(pts_scale["Substanzkennzahl"], errors="coerce")
        else:
            pts_metric = pd.to_numeric(pts_scale["Zustandsnote"], errors="coerce")

        rng = safe_minmax(pts_metric)
        if rng is None:
            rng = safe_minmax(gplot[metric_col])

        vmin, vmax = rng if rng is not None else (None, None)

        gplot_valid = gplot[gplot[metric_col].notna()]
        gplot_missing = gplot[gplot[metric_col].isna()]

        if len(gplot_missing) > 0:
            fig.add_trace(
                go.Choropleth(
                    geojson=geojson_kreise,
                    locations=gplot_missing["id"],
                    z=[0] * len(gplot_missing),
                    featureidkey="properties.id",
                    name="No data",
                    showlegend=True,
                    showscale=False,
                    colorscale=[[0, "lightgrey"], [1, "lightgrey"]],
                    hoverinfo="skip",
                    marker_line_color="rgba(0,0,0,0)",
                    marker_line_width=0.0,
                )
            )

        if len(gplot_valid) > 0:
            fig.add_trace(
                go.Choropleth(
                    geojson=geojson_kreise,
                    locations=gplot_valid["id"],
                    z=gplot_valid[metric_col],
                    featureidkey="properties.id",
                    name=f"{metric_label} (counties)",
                    showlegend=True,
                    showscale=True,
                    colorbar_title=(
                        f"{metric_label}<br>min={vmin:{fmt}} / max={vmax:{fmt}}"
                        if (vmin is not None and vmax is not None)
                        else metric_label
                    ),
                    #colorscale="RdYlGn_r",
                    colorscale=[[0.0, "#1E22FC"], [0.35, "lightgray"], [0.55, "#FC911E"], [0.70, "darkorange"], [1.0, "red"]],
                    zmin=vmin,
                    zmax=vmax,
                    marker_opacity=1.0,
                    hovertemplate=f"<b>%{{text}}</b><br>{metric_label}: %{{z:{fmt}}}<extra></extra>",
                    text=gplot_valid["NAME"],
                    marker_line_color="rgba(0,0,0,0)",
                    marker_line_width=0.0,
                )
            )

    if "states" in selected_layers:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_states,
                locations=gdf_states["state_id"],
                z=[0] * len(gdf_states),
                featureidkey="properties.state_id",
                name="States",
                showlegend=True,
                showscale=False,
                hoverinfo="skip",
                colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
                marker_line_color="rgba(70,70,70,70.95)",
                marker_line_width=1.3,
            )
        )

    if "kreise" in selected_layers:
        fig.add_trace(
            go.Choropleth(
                geojson=geojson_kreise,
                locations=gplot["id"],
                z=[0] * len(gplot),
                featureidkey="properties.id",
                name="County borders",
                showlegend=True,
                showscale=False,
                hoverinfo="skip",
                colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],
                marker_line_color="rgba(170,170,170,0.9)",
                marker_line_width=0.4,
            )
        )

    pts = gdf_pts.copy()

    if selected_baustoff:
        pts = pts[pts["Baustoffklasse"].isin(selected_baustoff)]

    if selected_year_range is not None and isinstance(selected_year_range, (list, tuple)) and len(selected_year_range) == 2:
        year_min, year_max = selected_year_range
        pts = pts[(pts["Baujahr"].notna()) & (pts["Baujahr"] >= year_min) & (pts["Baujahr"] <= year_max)]

    if selected_road:
        pts = pts[pts["RoadTypeLabel"].isin(selected_road)]

    if len(pts) > 0:
        pts_zust = pd.to_numeric(pts["Zustandsnote"], errors="coerce")
        zmin_pts, zmax_pts = safe_minmax_with_fallback(pts_zust, fallback=(1.0, 4.0))

        fig.add_trace(
            go.Scattergeo(
                lon=pts["lon"],
                lat=pts["lat"],
                mode="markers",
                name="Bridges (filtered)",
                text=pts["hover"],
                hovertemplate="%{text}<extra></extra>",
                marker=dict(
                    size=3.0,
                    opacity=0.85,
                    color=pts_zust,
                    cmin=zmin_pts,
                    cmax=zmax_pts,
                    colorscale="RdYlGn_r",
                    showscale=False,
                    colorbar=dict(title="Condition<br>(green = good / red = bad)"),
                    line=dict(color="rgba(0,0,0,1)", width=0.25),
                ),
                showlegend=True,
            )
        )

    fig.update_geos(
        fitbounds="locations",
        visible=False,
        projection_type="mercator",
        center=dict(lat=51, lon=10),
        projection_scale=7,
    )

    mapping_label = used_coords if spatial_join_ok else "Name match"
    fig.update_layout(
        margin={"r": 20, "t": 60, "l": 20, "b": 20},
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
        title=(
            "Average metrics per county (condition / load / age / substance)<br>"
            f"<sup>Mapping: {mapping_label}; Scale: min/max from filtered bridges</sup>"
        ),
    )

    return fig

## Plot for Report

In [17]:
def plot_germany_mean_condition_matplotlib(
    gplot,
    geojson_kreise,
    geojson_states,
    output_path="../plots_for_report/germany_mean_bridge_condition_matplotlib.pdf"
):

    # set color stylesheet
    plt.rcParams.update(bundles.icml2024(column="half", nrows=2, ncols=1))

    # create geodataframes
    gdf_kreise = gpd.GeoDataFrame.from_features(geojson_kreise["features"])
    gdf_states = gpd.GeoDataFrame.from_features(geojson_states["features"])

    land_union = gdf_kreise.unary_union.buffer(0.01)
    gdf_states_clipped = gdf_states.copy()
    if gdf_states.crs != gdf_kreise.crs:
        gdf_states = gdf_states.to_crs(gdf_kreise.crs)
    gdf_states_clipped['geometry'] = gdf_states_clipped.geometry.intersection(land_union)
    gdf_states_clipped = gdf_states_clipped[~gdf_states_clipped.is_empty]

    # merge data
    gdf_kreise = gdf_kreise.merge(gplot[["id", "mean_zust"]],
                                  left_on="id", right_on="id",
                                  how="left")

    # color scale
    cmap2 = mcolors.LinearSegmentedColormap.from_list(
        "custom",
        [(0.0, "#1E22FC"),
         (0.35, "lightgray"),
         (0.55, "#FC911E"),
         (0.70, "darkorange"),
         (1.0, "red")]
    )
    cmap = mcolors.LinearSegmentedColormap.from_list(
        "custom",
        [(0.0, "navy"),
         (0.25, "#585BFF"),
         (0.375, "lightgray"),
         (0.45, "#FC911E"),
         (0.6, "darkorange"),
         (1.0, "darkred")]
    )
  
    norm = mcolors.Normalize(vmin=1, vmax=4)

    fig, ax = plt.subplots()

    # plot districts
    gdf_kreise.plot(
        column="mean_zust",
        cmap=cmap,
        norm=norm,
        linewidth=0.3,
        edgecolor="black",
        ax=ax
    )

    # state border widths
    state_line_width_map = {
        "15": 1.6,
        "16": 1.6,
    }
    default_width = 0.8

    # assign widths based on state name
    gdf_states["line_width"] = gdf_states["state_id"].map(state_line_width_map).fillna(default_width)

    # plot state borders
    gdf_states_clipped.boundary.plot(
        ax=ax,
        linewidth=gdf_states["line_width"],
        edgecolor="black"
    )

    # add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    cbar = fig.colorbar(sm, ax=ax, fraction=0.04)
    cbar.ax.set_title("Mean\nCondition\nScore", pad=10)

    # adjust axes
    ax.set_axis_off()
    ax.set_aspect("auto")

    # save as pdf
    plt.savefig(output_path)
    plt.close()

plot_germany_mean_condition_matplotlib(gplot=gplot,
                                       geojson_kreise=geojson_kreise,
                                       geojson_states=geojson_states)

/var/folders/zj/y_y97nk968x1fwv19z05cpf80000gn/T/ipykernel_97936/3217032290.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  land_union = gdf_kreise.unary_union.buffer(0.01)


## Dash app layout

In [17]:
app = dash.Dash(__name__)

app.layout = html.Div(
    style={"display": "flex", "height": "100vh", "fontFamily": "sans-serif"},
    children=[
        html.Div(
            style={
                "width": "22%",
                "padding": "10px",
                "borderRight": "1px solid #ccc",
                "overflowY": "auto",
            },
            children=[
                html.H3("Layers"),
                dcc.Checklist(
                    id="layer-checklist",
                    options=layer_options,
                    value=[o["value"] for o in layer_options],
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.H3("Choropleth metric"),
                dcc.RadioItems(
                    id="choropleth-mode",
                    options=choropleth_mode_options,
                    value="zustand",
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.H3("Material class"),
                dcc.Checklist(
                    id="baustoff-checklist",
                    options=baustoff_options,
                    value=[],
                    labelStyle={"display": "block"},
                ),
                html.Hr(),
                html.H3("Year (superstructure)"),
                dcc.RangeSlider(
                    id="year-slider",
                    min=BAUJAHR_MIN,
                    max=BAUJAHR_MAX,
                    step=1,
                    value=[BAUJAHR_MIN, BAUJAHR_MAX],
                    marks=baujahr_marks,
                    allowCross=False,
                    tooltip={"always_visible": False, "placement": "bottom"},
                ),
                html.Hr(),
                html.H3("Road type"),
                dcc.Checklist(
                    id="road-checklist",
                    options=road_options,
                    value=[],
                    labelStyle={"display": "block"},
                ),
            ],
        ),
        html.Div(
            style={"width": "78%", "padding": "10px"},
            children=[
                dcc.Graph(id="map-figure", style={"height": "100%"})
            ],
        ),
    ],
)

## Callback + run server

In [18]:
@app.callback(
    Output("map-figure", "figure"),
    [
        Input("layer-checklist", "value"),
        Input("choropleth-mode", "value"),
        Input("baustoff-checklist", "value"),
        Input("year-slider", "value"),
        Input("road-checklist", "value"),
    ],
)
def update_map(selected_layers, choropleth_mode, selected_baustoff, selected_year_range, selected_road):
    if selected_layers is None:
        selected_layers = []
    if selected_baustoff is None:
        selected_baustoff = []
    if selected_year_range is None:
        selected_year_range = [BAUJAHR_MIN, BAUJAHR_MAX]
    if selected_road is None:
        selected_road = []
    if choropleth_mode is None:
        choropleth_mode = "zustand"

    return build_figure(
        selected_layers,
        selected_baustoff,
        selected_year_range,
        selected_road,
        choropleth_mode,
    )


app.run(debug=True, port=8071)
